# Curs 4: Pandas - elemente avansate

## Lucrul cu valori lipsa in Pandas

### Reprezentarea valorilor lipsa in Pandas

Pandas foloseste doua variante pentru reprezentarea de valori lipsa: None si NaN. NaN este utilizat pentru tipuri numerice in virgula mobila. None este convertit la NaN daca seria este numerica; daca seria este ne-numerica, se considera de tip `object`:

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
print(f'pandas version: {pd.__version__}')
print(f'numpy version: {np.__version__}')

# pandas version: 0.24.1
# numpy version: 1.16.2

NaN si None sunt echivalene in context numeric, in Pandas:

In [ ]:
pd.Series([1, np.nan, 2, None])

In [ ]:
pd.Series(['John', 'Danny', None])

Intrucat doar tipurile numerice floating point suporta valoare de NaN, conform standardulului IEEE 754, se va face transformarea unei serii de tip intreg intr-una de tip floating point daca se insereaza sau adauga un NaN:

In [ ]:
# creare de serie cu valori intregi
x = pd.Series([10, 20], dtype=int)
x

In [ ]:
x[1] = np.nan
x

In [ ]:
# adaugare cu append 
x = pd.Series([10, 20], dtype=int)
print(f'Serie de intregi:\n{x}')
x = x.append(pd.Series([100, np.nan]))
print(f'Dupa adaugare:\n{x}')

### Operatii cu valori lipsa in Pandas

Metodele ce se pot folosi pentru operarea cu valori lipsa sunt:
* `isnull()` - genereaza o matrice de valori logice, ce specifica daca pe pozitiile corespunzatoare sunt valori lipsa
* `nonull()` - complementara lui `isnull()`
* `dropna()` - returneaza o versiune filtrata a datelor, doar acele linii si coloane care nu au null
* `fillna()` - returneaza o copie a obiectului initial, in care valorile lipsa sunt umplute cu ceva specificat

#### `isnull()` si `nonull()`

In [ ]:
data = pd.Series([1, np.nan, 'hello', None])
data

In [ ]:
data.isnull()

Selectarea doar acelor valori din obiectul Series care sunt ne-nule se face cu:

In [ ]:
# filtrare
data[data.notnull()]

Functiile `isnull()` si `notnull()` functioneaza la fel si pentru obiecte DataFrame:

In [ ]:
df = pd.DataFrame({'Name': ['Will', 'Mary', 'Joan'], 'Age': [20, 25, 30]})
df

In [ ]:
df.loc[2, 'Age'] = np.NaN
df

In [ ]:
df.isnull()

In [ ]:
df.notnull()

In cazul obiectelor DataFrame, aplicarea lui `notnull()` nu lasa afara elemente din dataframe:

In [ ]:
df[df.notnull()]

#### Stergerea de elemente cu `dropna()`

Pentru un obiect Series, metoda `dropna()` produce un alt obiect in care liniile cu valori de null sunt sterse:

In [ ]:
data

In [ ]:
data2 = data.dropna()
data2

Pentru un obiect DataFrame se pot sterge doar linii sau coloane intregi - obiectul care ramane trebuie sa fie tot un DataFrame:

In [ ]:
df = pd.DataFrame([[1, np.nan, 2],
[2, 3, 5],
[np.nan, 4, 6]])
df

In [ ]:
# Implicit: eliminare de linii care contin null
df2 = df.dropna()
df2

Mai sus s-a ales implicit stergerea de linii, datorita faptului ca parametrul `axis` are implicit valoarea 0:

In [ ]:
help(df.dropna)

Se poate opta pentru stergerea de coloane care contin null:

In [ ]:
df

In [ ]:
# stergere de coloane cu null
# df3 = df.dropna(axis=1) # functioneaza
df3 = df.dropna(axis='columns')
df3

Operatiile de mai sus sterg o linie sau o coloana daca ea contine cel putin o valoare de null. Se poate cere stergerea doar in cazul in care intreaga linie sau coloana e plina cu null, folosind parametrul `how`:

In [ ]:
df

In [ ]:
df2 = df.dropna(how='all')
df2

De remarcat ca `dropna()` nu modifica obiectul originar, decat daca se specifica paarametrul `inplace=True`. 

#### Umplerea de valori nule cu `fillna()`

In [ ]:
data = pd.Series([1, np.nan, 2, None, 3], index=list('abcde'))

In [ ]:
# umplere cu valoare constanta
data2 = data.fillna(0)
data2

In [ ]:
# Umplere cu copierea ultimei valori cunoscute:
data2 = data.fillna(method='ffill')
data2

In [ ]:
# Umplere 'inapoi':
data2 = data.fillna(method='bfill')
data2

In [ ]:
# umplerea cu valoare calculata:
print(f'Media valorilor non-nan este: {data.mean()}')
data2 = data.fillna(data.mean())
data2

## Agregare si grupare

### Agregari simple

In [ ]:
np.random.seed(100)
ser = pd.Series(np.random.rand(10))
ser

In [ ]:
ser.sum(), ser.max(), ser.min()

Pentru obiecte DataFrame, operatiile de agregare opereaza pe coloane:

In [ ]:
df = pd.DataFrame({'A': np.random.rand(10), 'B': -np.random.rand(10) }, index=['line ' + str(i) for i in range(1, 11)])
df

In [ ]:
df.mean()

.. si daca se doreste calculul pe linii, se poate indica via parametrul `axis`:

In [ ]:
# df.mean(axis=1)
df.mean(axis='column')

Exista o metoda utila, care pentru un obiect DataFrame aclculeaza statisticile:

In [ ]:
df.describe()

Operatiile nu iau in considerare valorile lipsa:

In [ ]:
df.iloc[0, 0] = df.iloc[0,1] = np.nan
df.iloc[5, 0] = df.iloc[7, 1] = df.iloc[9, 1] = np.nan
df

In [ ]:
df.count()


|  Metoda de agregare | Descriere  |
|---|---|
|  count() | Numarul total de elemente   |
|  first(), last() | primul si ultimul element  |
|  mean(), median() | Media si mediana  |
|  min(), max() | Minimul si maximul  |
|  std(), var()  | Deviatia standard si varianta  |
|  mad() | Deviatia absoluta medie  |
|  prod(), sum() | Produsul si suma elementelor  |

### Gruparea datelor: `split()`, `apply()`, `combine()`

Pasii care se fac pentru agregarea datelor urmeazaz secventa: imparte, aplica operatie, combina:
1. imparte - via metoda `split()`: separa datele initiale in grupuri, pe baza unei chei
1. aplica, via metoda `apply()`: calculeaza o functie pentru fiecare grup: agregare, transformare, filtrare
1. combina, via metoda `combine()`: concateneaza rezultatele si rpodu raspunsul final

![Apply-split-combine](./images/asc.png)

In [ ]:
df = pd.DataFrame({'key': ['A', 'B', 'C', 'A', 'B', 'C'], 'data': range(6)}, columns=['key', 'data'])
df

In [ ]:
groups = df.groupby('key')
type(groups)

In [ ]:
print(groups)

In [ ]:
groups.sum()

Ca functie de agregare se poate folosi orice functie Pandas sau NumPy.

In [ ]:
import seaborn as sns
planets = sns.load_dataset('planets')

In [ ]:
planets.head()

In [ ]:
# planets.describe()

In [ ]:
planets.method.unique()

Pentru grupurile rezultate se poate alege o coloana, pentru care sa se calculeze valori agregate:

In [ ]:
planets.groupby('method')['orbital_period'].median()

Grupurile pot fi iterate, returnand pentru fiecare grup un obiect de tip Series sau DataFrame:

In [ ]:
print(f'Number of columns: {len(planets.columns)}')

for (method, group) in planets.groupby('method'):
    print("{0:30s} shape={1}".format(method, group.shape))

Fiecare grup rezultat, fiind vazut ca un Series sau DataFrame, suporta apel de metode aferete acestor obiecte:

In [ ]:
planets.groupby('method')['year'].describe()

### Metodele `aggregate()`, `filter()`, `transform()`, `apply()`

Inainte de pasul de combinare a datelor se pot folosi metode care implementeaza operatii pe grupurim inainte de a face in final gruparea rezultatelor din grupuri.

In [ ]:
df = pd.DataFrame({'key': ['A', 'B', 'C', 'A', 'B', 'C'],
'data1': range(6),
'data2': np.random.randint(0, 10, 6)},
columns = ['key', 'data1', 'data2'])
df

Metoda `aggregate()` permite specificare de functii prin numele lor (string sau referinta la functie):

In [ ]:
df.groupby('key').aggregate(['min', np.median, max])

Filtrarea cu `filter()` permite selectarea doar acelor grupuri care satisfac o anumita conditie:

In [ ]:
def filter_func(x): # x este o linie, corespunzand fiecarui grup
    return x['data2'].std() > 4

In [ ]:
df.groupby('key').std()

In [ ]:
df.groupby('key').filter(filter_func)

Acelasi efect se obtine cu lambda functii:

In [ ]:
df.groupby('key').filter(lambda row: row['data2'].std() > 4)

Transformarea cu `transform()` produce un dataframe cu acelasi numar de linii ca si cel initial, dar cu valorile calculate prin aplicarea unei operatii la nivelul fiecarui grup:

In [ ]:
df

Media pe fieare grup este:

In [ ]:
df.groupby('key').mean()

Centrarea valorilor pentru fiecare grup - adica: in fiecare grup sa fie media 0 - se face cu:

In [ ]:
df.groupby('key').transform(lambda x: x - x.mean())

In [ ]:
df.groupby('key').transform(lambda x: x - x.mean()).mean()

Functia `apply()` permite calculul unei functii peste fiecare grup. Exemplul de mai jos calculeaza prima coloana impartita la suma elementelor din coloana data2, in cadrul fiecarui grup: 

In [ ]:
def norm_by_data2(x):
    # x is a DataFrame of group values
    x['data1'] /= x['data2'].sum()
    return x

df.groupby('key').apply(norm_by_data2)

Functia `apply()` se poate folosi si in afara lui `groupby`, permitand calcul vectorizat de mare viteza:

In [ ]:
data_len = 10000
# df_big = pd.DataFrame({'Noise_1': np.random.rand(data_len), 'Noise_2': np.random.rand(data_len), 'Noise_3': np.random.rand(data_len)})

df_big = pd.DataFrame({'Noise_' + str(i) : np.random.rand(data_len) for i in range(1, 50)})

df_big.head()

In [ ]:
all_noise_columns = [column for column in df_big.columns if column.startswith('Noise_')]

row = df_big.iloc[0]
row[all_noise_columns] 

In [ ]:
np.mean(row[all_noise_columns]) > 0.1

In [ ]:
%%timeit

df_big['All_noises'] = df_big.apply(lambda row: np.mean(row[all_noise_columns]) > 0.1, axis=1)

# 11 s ± 592 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)?

In [ ]:
%%timeit

for index in df_big.index:
    df_big.loc[index, 'All_noises'] = np.mean(df_big.loc[index, all_noise_columns]) > 0.1   
#     22.5 s ± 592 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

## Tabele pivot

!['Titanic'](./images/titanic.jpg)

In [ ]:
# Incarcarea datelor:

titanic = sns.load_dataset('titanic')
titanic.head()

Pornim de la urmatoarea problema: care este procentul de femei si barbati supravietuitori? Diferentierea de gen se face dupa coloana 'sex', iar supravietuirea este in coloana 'survived':

In [ ]:
titanic.groupby('sex')['survived'].mean()

Mai departe, se cere determinarea distributiei pe gen si clasa imbarcare, folosind `groupby()`:

In [ ]:
titanic.groupby(['sex', 'class'])['survived'].aggregate('mean').unstack()

Acest tip de operatii (grupare dupa doua atribute, calcul de valori agregate) este des intalnit si se numeste pivotare. Pandas introduce suport nativ pentru pivotare, simplificand codul:

In [ ]:
titanic.pivot_table('survived', index='sex', columns='class' )

Se poate face pivotare pe mai mult de doua niveluri (mai sus: sex si class). De exemplu, varsta poate fi adaugata pentru analiza, persoane sub 18 ani (copii) si cei peste 18 (adulti). In primul pas se poate face impartirea persoanelor pe cele doua subintervale de varsta (<=18, >18) folosind `cut`:

In [ ]:
age = pd.cut(titanic['age'], [0, 18, 80], labels=['child', 'adult'])
age.head(15)

In [ ]:
titanic.pivot_table('survived', ['sex', age], 'class')

In [ ]:
fare_split = pd.cut(titanic.fare, 2, labels=['cheap fare', 'expensive fare'])

In [ ]:
fare_split

In [ ]:
titanic.pivot_table('survived', ['sex', age, fare_split], 'class')